# 二、	條件式的自動化選股系統設計

結合統計觀念與技術指標，設計一自動化選股系統，利用爬取之股票數據計算並分析，提供投資人購買該支股票與否的資訊與建議。

In [31]:
# 載入所需套件

import requests
from bs4 import BeautifulSoup
import json
import datetime
import time
import pandas as pd

In [32]:
###############################################################################
#                     股票機器人 標準差＆本益比分析選股                            #
###############################################################################

# 抓到今天的時間
now = datetime.datetime.now()

# 先與網站請求抓到價格
def getstock(stocknumber):
    
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20180814&stockNo=' + stocknumber + '&_=' + str(time.mktime(now.timetuple()))
    
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content, "html.parser")
    getjson=json.loads(soup.text)
    
    # 判斷請求是否成功
    if getjson['stat'] != '很抱歉，沒有符合條件的資料!': 
        return [getjson['data']]
    else:
        return [] # 請求失敗回傳空值

In [33]:
# 開始計算股票的平均以及標準差＆抓取指定股票之本益比，並從參考數值判斷是否購買

def Standard_Deviation(stocknumber):    
    # 第一步 載入所需套件 
    import requests
    import pandas as pd
    import numpy as np
    from io import StringIO
    # 第二步 爬取股票資訊（含本益比）
    stocklist = getstock(stocknumber)
    date = str(now.date())
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date + '&type=ALL')

    # 第三步 篩選出個股盤後資訊
    str_list = []
    for i in r.text.split('\n'):
        if len(i.split('",')) == 17 and i[0] != '=':       
            i = i.strip(",\r\n")
            str_list.append(i)      

    # 第四步 印出選股資訊
    df = pd.read_csv(StringIO("\n".join(str_list)))  
    pd.set_option('display.max_rows', None)
    
    n = len(df['證券代號'])
    for i in range(n):
        if df['證券代號'][i]==str(stocknumber):
            print("該支股票今日本益比為：",df['本益比'][i],"\n")
    
    # 判斷是否為空值
    if len(stocklist) != 0:
        stockdf = pd.DataFrame(stocklist[0],columns=["日期","成交股數","成交金額","開盤價","最高價","最低價",
                                                     "收盤價","漲跌價差","成交筆數"])
        stockAverage = pd.to_numeric(stockdf['收盤價']).mean()
        stockSTD = pd.to_numeric(stockdf['收盤價']).std()
        
        # 看看這隻股票現在是否便宜（平均-1.5倍標準差）
        #buy='\n很貴不要買'
        for i in range(n):
            if df['證券代號'][i]==str(stocknumber):
                if (float(df['本益比'][i])<=12):
                    buy='\n本益比小於12'
                    if pd.to_numeric(stockdf['收盤價'][-1:]).values[0] < stockAverage - (1.5*stockSTD):
                        buy += '，而且這隻股票現在很便宜喔！'
                    else:
                        buy+= '，但很貴不要買'
                else:        
                     buy='本益比大於12'
        # 顯示結果
        print('收盤價 ＝ ' + stockdf['收盤價'][-1:].values[0])
        print('中間價 ＝ ' + str(stockAverage))
        print('線距 ＝ ' + str(stockSTD))
        #print(PE,end='')
        print(buy)
        j=0
    else:
        print('請求失敗，請檢查您的股票代號')
    return df

In [34]:
Standard_Deviation(stocknumber='2330')


該支股票今日本益比為： 24.65 

收盤價 ＝ 256.00
中間價 ＝ 245.80434782608697
線距 ＝ 6.249980237122904
本益比大於12


,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,1101,台泥,"6,197,147","3,078","258,627,565",41.70,41.85,41.55,41.75,+,0.10,41.70,211,41.75,437,10.28
1,1101B,台泥乙特,"3,000",3,"160,900",53.60,53.70,53.60,53.70,,0.00,53.40,2,53.70,5,0.00
2,1102,亞泥,"9,053,787","3,195","419,001,570",45.80,46.50,45.70,46.45,+,0.65,46.40,1,46.45,4,10.23
3,1103,嘉泥,"2,378,464",965,"46,531,542",19.70,19.75,19.40,19.45,-,0.35,19.45,109,19.50,22,9.63
4,1104,環泥,"281,730",148,"5,289,108",18.70,18.80,18.70,18.75,+,0.05,18.75,43,18.80,30,12.25
5,1108,幸福,"44,000",21,"336,140",7.65,7.65,7.61,7.64,+,0.04,7.61,1,7.64,2,0.00
6,1109,信大,"547,624",254,"10,089,331",18.50,18.60,18.30,18.45,-,0.05,18.45,9,18.50,31,6.45
7,1110,東泥,"11,000",6,"186,750",17.00,17.00,16.95,17.00,,0.00,16.90,1,17.00,37,850.00
8,1201,味全,"1,292,171",707,"31,959,727",25.00,25.00,24.65,24.65,-,0.15,24.65,23,24.70,53,9.94
9,1203,味王,"33,467",22,"901,711",27.10,27.10,26.85,27.00,,0.00,26.90,2,27.00,15,14.67
